Let's load up our data and do a little cleanup. We have a column that is always null and another that is redundant, so we can drop them both.

In [1]:
import pandas as pd

DATA = '/kaggle/input/meteorite-landings-on-earth-data/Meteorite_Landings.csv'
df = pd.read_csv(filepath_or_buffer=DATA).drop(columns=['Unnamed: 10', 'GeoLocation'])
df.head()

,name,id,nametype,recclass,mass (g),fall,year,reclat,reclong
0,Aachen,1,Valid,L5,21.0,Fell,1880.0,50.77500,6.08333
1,Aarhus,2,Valid,H6,720.0,Fell,1951.0,56.18333,10.23333
2,Abee,6,Valid,EH4,107000.0,Fell,1952.0,54.21667,-113.00000
3,Acapulco,10,Valid,Acapulcoite,1914.0,Fell,1976.0,16.88333,-99.90000
4,Achiras,370,Valid,L6,780.0,Fell,1902.0,-33.16667,-64.95000


Let's take a quick look at a couple of columns that only have two different values.

In [2]:
df['nametype'].value_counts().to_dict()

{'Valid': 45641, 'Relict': 75}

In [3]:
df['fall'].value_counts().to_dict()

{'Found': 44609, 'Fell': 1107}

Let's start by making some exploratory graphs.

In [4]:
from plotly import express
from plotly import io

io.renderers.default = 'iframe'
express.bar(data_frame=df['recclass'].value_counts().to_frame().reset_index().head(n=40), x='recclass', y='count')

Okay that's interesting. Our dataset is dominated by eight classes, and then there's a long tail.

Let's take a look at the mass data; how do we expect it to be distributed? There are probably meteors too small to be noticed, but also a practical limit on how large a meteor could be, so we might expect a normal distribution. 

In [5]:
from warnings import filterwarnings

filterwarnings(action='ignore', category=RuntimeWarning)
express.histogram(data_frame=df[df['mass (g)'] < 100], x='mass (g)')

That's not a normal distribution, but it's definitely got only one mode. We had to cut off the right tail for clarity but we didn't really lose any thing crucial to the meaning of this graph by doing so.

We have some annual data, so let's take a look at that too.

In [6]:
express.histogram(data_frame=df[(df['year'] > 1850) & (df['year'] < 2025)], x='year')

That's an interesting plot. Do we believe that there were almost no meteors prior to 1974 and then suddenly there were thousands per year? No, it seems more likely there was sudden change in people looking for meteors and the methods they used.

We knew this moment was coming; let's make a map.

In [7]:
express.scatter_mapbox(data_frame=df[(df['year'] > 1970) & (df['year'] < 2025)], lat='reclat', lon='reclong', color='year',
                       mapbox_style='open-street-map', height=800, zoom=1, center={'lat': 0, 'lon': 0}, hover_name='name')

Interestingly, there are parts of the world where we don't know much about their meteors until very recently.

Let's take another look, but color by the mass.

In [8]:
express.scatter_mapbox(data_frame=df[df['mass (g)'] < 1000], lat='reclat', lon='reclong', color='mass (g)',
                       mapbox_style='open-street-map', height=800, zoom=1, center={'lat': 0, 'lon': 0}, hover_name='name')

The distribution of mass doesn't show a clear geographic pattern, although maybe if we squint we can see that smaller meteors are more prevalent near populated areas.